# 导入依赖
导入数据分析、可视化与建模所需的常用库。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# 数据读取与初探
- 读取 `sf-crime/train.csv`
- 查看前几行与缺失值统计

In [ ]:

df = pd.read_csv('sf-crime/train.csv')

# A. Data Understanding and Preprocessing

# 1. Check the first few rows of the dataset
df.head()

# 2. Check for missing values
df.isnull().sum()


# 数据清洗
- 解析并清洗日期列（无法解析的记录将被丢弃）
- 去重
- 过滤经纬度异常（限定在旧金山经纬度范围）
- 打印清洗后数据的形状

In [ ]:
# 3. Basic structure and types
print('Shape:', df.shape)
print(df.dtypes)

# 4. Parse date column and handle invalid values
date_col = 'Dates' if 'Dates' in df.columns else None
if date_col:
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    invalid_dates = df[date_col].isna().sum()
    if invalid_dates:
        print(f'Invalid dates: {invalid_dates} — dropping them')
        df = df[df[date_col].notna()].copy()
else:
    print('No date column found')

# 5. Drop exact duplicates
dup_count = df.duplicated().sum()
print('Duplicate rows:', dup_count)
if dup_count:
    df = df.drop_duplicates().copy()

# 6. Sanity check coordinates (keep plausible SF bounds)
if {'X','Y'}.issubset(df.columns):
    before = len(df)
    df = df[df['Y'].between(37.0, 38.0) & df['X'].between(-123.0, -121.0)].copy()
    print(f'Removed {before - len(df)} rows outside SF bounds')

print('Cleaned shape:', df.shape)


# 特征工程与编码
- 构造 Arrest_Indicator、Year/Month/Day/Hour、Incident_Quarter
- 对 DayOfWeek、PdDistrict、Resolution、Address 进行onehot编码；若存在 Category，则额外标签编码
- 示例过滤：按年份（2015）与分局（Mission）生成子集

In [ ]:
# 7. Derived variables: Arrest Indicator, Incident Quarter, calendar parts
if 'Resolution' in df.columns:
    df['Arrest_Indicator'] = df['Resolution'].str.contains('ARREST', case=False, na=False).astype(int)

if 'Dates' in df.columns and pd.api.types.is_datetime64_any_dtype(df['Dates']):
    df['Year'] = df['Dates'].dt.year
    df['Month'] = df['Dates'].dt.month
    df['Day'] = df['Dates'].dt.day
    df['Hour'] = df['Dates'].dt.hour
    df['Incident_Quarter'] = df['Dates'].dt.to_period('Q').astype(str)

# 8. Encode categorical variables (one-hot)
categorical_cols = [c for c in ['DayOfWeek','PdDistrict','Resolution','Address'] if c in df.columns]
for c in categorical_cols:
    df[c] = df[c].astype('category')

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Optionally encode target for modeling later
if 'Category' in df_encoded.columns:
    le = LabelEncoder()
    df_encoded['Category_encoded'] = le.fit_transform(df_encoded['Category'])

print('Encoded shape:', df_encoded.shape)
display(df_encoded.head())

# 9. Optional filter examples (by year and district)
if 'Year' in df_encoded.columns:
    df_2015 = df_encoded[df_encoded['Year'] == 2015].copy()
else:
    df_2015 = df_encoded.copy()

if 'PdDistrict' in df.columns:
    mission_idx = df['PdDistrict'] == 'MISSION'
    df_mission = df_encoded[mission_idx].copy()
else:
    df_mission = df_encoded.copy()

print('Rows in 2015:', len(df_2015))
print('Rows in Mission district:', len(df_mission))
